# XCS224N Natural Language Processing with Deep Learning


# Lecture 2

[CS224N](http://web.stanford.edu/class/cs224n/) / [XCS224N](http://scpd.stanford.edu/search/publicCourseSearchDetails.do?method=load&courseId=93933715) / [Lecture](https://youtu.be/kEMJRjEdNzM) / [Slides](http://web.stanford.edu/class/cs224n/slides/cs224n-2019-lecture02-wordvecs2.pdf)

## More on Word2Vec

Reviewing Word2Vec once more...

<img src="images/word2vec_example.PNG" />

$$P(o|c) = \frac{exp(u_{O}^{T}v_{c})}{\sum_{w\in V}exp(u_{W}^{T}v_{c})}$$

### Word2Vec parameters and computations

In most libraries, such as TensorFlow and PyTorch, word vectors are stored as rows, contrary to being stored as columns as seen in linear algebra.

<img src="images/matrices.PNG" />

The output from Word2Vec is a probability distribution for all words, assigning reasonably high probabilities to words that often occur in the context of a given center word.

However, it is important to consider common words such as <strong>"the"</strong>, <strong>"and"</strong>, and so on. They are very common as context words and will have a high dot product value. This can be referred to as the <strong>high frequency probability effect</strong>.

### Word2Vec maximizes objective function by putting similar words nearby in space

<img src="images/word_vector_example.PNG" />

There is a drawback to these visualisations though. Words can be close to one another in many different directions (across multiple dimensions). This is where 2 dimensional projections can be misleading, and may only show a small part of the relationships between words.

## Optimization Basics

### Gradient Descent

We have a cost function $J(\theta)$ we would like to minimize 

Utilizing <strong>gradient descent</strong> we can minimize $J(\theta)$

The concept is:
* For the current $\theta$, calculate the gradient of $J(\theta)$
* Then take a small step in the direction of the negative gradient (i.e. walking down the hill)
* Repeat until optimized

<img src="images/gradient_descent.PNG" />

Updating the equation (in matrix notation) is done by via the step size or learning (<strong>$\alpha$</strong>):

$$\theta^{new} = \theta^{old} - \alpha\bigtriangledown_\theta J(\theta)$$

Updating the equation for a single parameter is done via the following equation:

$$\theta_{j}^{new} = \theta_{j}^{old} - \alpha \frac{\partial}{\partial\theta_{j}^{old}} J(\theta)$$

Algorithm:
```python 
while true:
    theta_grad = evaluate gradient(J, corpus, theta)
    theta = theta - alpha * theta_grad
```

### Stochastic Gradient Descent

For the example where you will likely need to optimize the gradient for billions of records, the approach of gradient desecent for all windows in the corpus becomes too expensive to compute.

<strong>Problem</strong>: $J(\theta)$ is a function of <strong>all</strong> windows, for example, in the billions in some cases depending on the training dataset. Thus, $\bigtriangledown_\theta J(\theta)$ can be too expensive to compute.

<strong>Solution</strong>: <strong>Stochastic gradient descent (SGD)</strong> by batching sample windows, and updating the cost function after each sample in the batch.

The approach for <strong>SGD</strong> with Word Vectors:
* Iterate taking gradients at each such window for SGD
* In each window, we only have 2m + 1 words, so $\bigtriangledown_\theta J_{t}(\theta)$

$\bigtriangledown_\theta J_{t}(\theta) =  
\begin{pmatrix}
  0 \\
  \vdots \\
  \bigtriangledown_{v_{like}} \\
  \vdots \\
  0 \\
  \bigtriangledown_{u_{I}} \\
  \vdots \\
  \bigtriangledown_{u_{learning}} \\
  \vdots
 \end{pmatrix}
 \in \mathbb{R}^{2dV}
 $

Just about all elements in the $\bigtriangledown_\theta J_{t}(\theta)$ vector are 0, in other words it is a <strong>sparse vector</strong>. This differs from other use cases in deep learning such as computer vision.

<strong>Problem</strong>: This poses the problem of very sparse parameter updating. Thus, we probably only want to update the non-zero vectors but this can be complex and may require matrix operations and strategies to accomodate distributed computation to get parameter estimates.

<strong>Solution</strong>:
* Sparse matrix update operations to only update the certain rows of full embedding matrics U and V
* Hash for word vectors

## Word2Vec: Model Variants

Why two vectors? Easier optimization, however it can be done with just one vector per word.

Two model variants:
1. Skip-grams (SG) - <em>The presented model here</em>
2. Continuous Bag of Words (CBOW): Predict center word from (bag of) context words

###  Skip-gram model with negative sampling

To get additional efficiency in training, we adopt <strong>negative sampling</strong>. So far, we have focused on <strong>naive softmax</strong> which is simpler and a more computationally expensive training method.

$$P(o|c) = \frac{exp(u_{O}^{T}v_{c})}{\sum_{w\in V}exp(u_{W}^{T}v_{c})}$$

The denominator or normalization factor requires you to sum over the entire vocabulary (i.e. all words in the vocabulary). This also means for however many words in the vocabulary we will need to compute the same amount of dot products. This is too computationally expensive, hence <strong>negative sampling (Mikolov and colleagues)</strong> was born.

The idea is to train binary logistic regressions for a true pair (center word and word in its context window) versus several noise pairs (the center word paired with a random word).
* Assigning high probabilities to words that were actually seen (true pairs)
* Assigning low probabilities to words (randomly sampled) that were not seen (noise pairs) 

“Distributed Representations of Words and Phrases and their Compositionality” (Mikolov et al. 2013)

Notation to be used for negative sampling in XCS224N (differing to the one in Mikolov et al. 2013):

$$J_{neg - sample}(o,v_{c},U) = -log(\sigma(u_{o}^{T}v_{c})) - \sum_{k=1}^{K}log(\sigma(-u_{k}^{T}v_{c}))$$

* Maximising probability of real outside words that appear i.e. in two words co-occurring with first log
* Penalising/minimizing probability that random words appear around center word i.e. taking k negative samples (using word probabilities)

$$P(w) = \frac{U(w)^{3/4}}{Z}$$

* Unigram distribution raised to the 3/4 power ensures common words are penalised and uncommon words maxmimized
* Z being the normalizing term
* w being the count of the word for every word in the vocabulary

## Alternative Methods

### Co-occurrence Matrix

Similarly to Word2vec, we use a window around each word to capture both semantic and syntactic (Part of Speech) meaning.

The result being a window-based co-occurence matrix with the following properties and problems:
* Symmetric
* Increase in size with vocabulary
* Very high dimensionality requiring a lot of storage
* Downstream classification models have sparsity issues
* Models are less robust

<br>

<img src="images/co-occurence.PNG" />

<strong>Solution</strong>: Store and preserve most of the informing variance in a fixed and smaller dense matrix using Dimensionality Reduction. Usually down to somewhere between 25-1000 dimensions (similar to Word2vec).

#### Singular Value Decomposition of the Co-occurence Matrix

- Latent semantic analysis

<img src="images/SVD.PNG" />

### Hacks to X

(Rohde et al. 2005)

Hacks included:
* For function words - $min(X, t)$, with $t \approx 100$
* For function words - ignore them all
* Ramped windows with more emphasis on closer words in the window
* Pearson correlations instead of counts, then set negative values to 0

The result being some fairly handy word vectors.

Some interesting results are generated with verbs and respective doers holding linear relatinships. The result being a model that has analogical significance.

<img src="images/semantic_similarity.PNG" />

### Count based vs. direct prediction

<img src="images/count_vs_predict.PNG" />

What about combining the two to get superior results?

<strong>Encoding meaning in vector differences [Pennington, Socher, and Manning, EMNLP 2014]</strong>

<img src="images/vector_difference.PNG" />

<strong>Q</strong> How can we capture ratios of co-occurence probabilities as linear meaning components in a word vector space?

<strong>A</strong>

Log-bilinear model: $$w_{i} . w_{j} = logP(i|j)$$

With vector differences: $$w_{x}.(w_{a} - w_{b}) = log(\frac{P(x|a)}{P(x|b)}$$

## GloVe
[Pennington et al., EMNLP 2014]

$$w_{i} . w_{j} = logP(i|j)$$

$$J = \sum_{i,j=1}^{V}f(X_{ij})(w_{i}^{T}\tilde{w}_{j}+b_{i}+\tilde{b}_{j} - log X_{ij})^{2} $$

Resulting in:
* Fast training
* Scalable to huge corpora
* Good performance even with small vectors and corpus
* $f$ function used to limit impact of common words also

## Evaluating Word Vectors

Generally in NLP, Intrinsic vs. Extrinsic evaluation is used.

<strong>Intrinsic</strong>

Are synonyms grouped together?

Fast to compute?

How does it evaluate on specific/intermediate subtasks? e.g. analogies

<strong>Extrinsic</strong>

Does it make the system better?

How long does it take to compute?

How does it perform on a real task?

### Intrinsic Word Vector Evaluation

Word Vector Analogies

$a:b :: c:?$

GloVe paper:
* Good dimensionality is ~300
* Window size 8 works best for GloVe

Dimensionality of Word Embeddings [Zi Yin and Yuanyuan Shen, NeurIPS 2018]
* Things don't fall apart at high dimensions contrary to other beliefs

Wikipedia data is superior to news and other data sources for training Word Embeddings.

Similarity judgements (based on word vector distances and their correlations with human judgements). May also use cosine similarity.

## Word Senses

Most words have lots of meanings and possess ambiguity (especially for common words or words that have existed for a long time).

How does a single vector capture multiple meanings?

Improving Word Representations Via Global Context And Multiple Word Prototypes (Huang et al. 2012)
* Breaking word into its pseudowords via clusters
* Then retrain with each word assigned to cluster i.e. $bank_{1}$, $bank_{2}$, etc

Linear Algebraic Structure of Word Senses, with Applications to Polysemy (Arora,.., Ma,..., TACL 2018)
* Difference senses of a word reside in a linear Superposition such as a Word2vec i.e. weighted average

### Extrinsic Word Vector Evaluation

Example: Finding a person, organization or location

By just using Word2vec, the efficiency or accuracy of your system improved dramatically.